In [60]:
import utils.torch_resistance_metrics as ermet
import utils.data_loader as loader
import torch
import os
import numpy as np
import pickle
from tqdm import tqdm

In [76]:
PATHS = [
'results/facebook_FOSR_weak_5000lnks_11_04_24__14_28_10/added_links.pt',
'results/google_FOSR_weak_5000lnks_11_04_24__14_28_35/added_links.pt',
'results/UNC28_FOSR_weak_5000lnks_11_04_24__14_32_13/added_links.pt'

]

In [77]:
for link_path in PATHS:
    output_path_folder = os.path.dirname(link_path)+os.sep

    DATASET = link_path.split('/')[1].split('_')[0]
    GW = loader.load_data(dataset = DATASET, device = 'cuda')
    GW.mode = 'woodbury'
    print(DATASET)

    added_edges = torch.load(link_path).T

    results_dict = {}
    unique_groups = GW.sens.unique().cpu().detach().numpy()
    for group in unique_groups:
        results_dict[group] = {}
        group_res = ermet.get_group_metrics(GW.get_effective_resistance().cpu().detach(),
                                                GW.sens.cpu().detach()==group,
                                                ~GW.edge_mask.cpu().detach())
        for metric in group_res:
            results_dict[group][metric] = [group_res[metric]]
    for u,v in tqdm(added_edges):
        if GW.is_edge(u,v):
            assert torch.Tensor([v,u]) in added_edges
        else:
            GW.add_link(u,v)
            Rcpu = GW.get_effective_resistance().cpu().detach()
            for group in unique_groups:
                group_res = ermet.get_group_metrics(Rcpu,
                                                    GW.sens.cpu().detach()==group,
                                                    ~GW.edge_mask.cpu().detach())
                for metric in group_res:
                    results_dict[group][metric].append(group_res[metric])


    #* Save dict as pickle
    with open(os.path.join(output_path_folder, 'results_dict.pkl'), 'wb') as f:
        pickle.dump(results_dict, f)

    #* Save effective resistance matrix
    np.save(os.path.join(output_path_folder, 'R.npy'), GW.get_effective_resistance().cpu().detach().numpy())
    


Loaded Graph with 1034 nodes and 26749 edges - L computation mode: exact
facebook


100%|██████████| 10000/10000 [01:49<00:00, 91.27it/s]


Loaded Graph with 3508 nodes and 253930 edges - L computation mode: exact
google


100%|██████████| 10000/10000 [16:14<00:00, 10.26it/s]


Loaded Graph with 3985 nodes and 65287 edges - L computation mode: exact
UNC28


100%|██████████| 10000/10000 [20:00<00:00,  8.33it/s]
